<a href="https://colab.research.google.com/github/mohisa302/Lane-detection/blob/master/Copy_of_detectingline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
from __future__ import division
import random
from keras.callbacks import History
from keras.layers import Dense, Activation, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, Dropout, BatchNormalization
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
import pickle
import csv
import cv2
import os
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt
from itertools import islice
import warnings
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import load_model
import keras.utils

#For Visualizations
from scipy.stats import norm, skew
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import division
import random,pickle,csv,cv2,os,scipy,pickle,warnings,matplotlib
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.stats import norm,skew
from itertools import islice
!pip install np_utils

import keras.backend as K
from keras.callbacks import History
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D,GlobalMaxPooling2D
from keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler
from tensorflow.keras.optimizers import Adam,SGD
from keras import applications
from keras.utils.vis_utils import plot_model

print(os.listdir('../content'))
warnings.filterwarnings('ignore')

     |████████████████████████████████| 61 kB 492 kB/s 
  Created wheel for np-utils: filename=np_utils-0.6.0-py3-none-any.whl size=56459 sha256=577deba6697ca018dc73d3e79f696195795b169c8ffc78c980cdaa5316df557b
  Stored in directory: /root/.cache/pip/wheels/d2/83/71/a781667865955ae7dc18e5a4038401deb56d96eb85d3a5f1c0
Successfully built np-utils
['.config', 'drive', 'data', 'self driving car training data', 'sample_data']


# New Section

In [ ]:
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [ ]:

from zipfile import ZipFile
zf = ZipFile('/content/drive/MyDrive/archive.zip', 'r')
zf.extractall('../content/')


In [ ]:
def image_preprocessing(img):
    resized_image = cv2.resize((cv2.cvtColor(img,cv2.COLOR_RGB2HSV))[:,:,1],(40,40))
    return resized_image

In [ ]:
import csv

features_directory = '../content/data/'
labels_file ='../content/data/driving_log.csv'

def Load_training(delta):
    logs = []
    features = []
    labels = []
    with open(labels_file, 'rt') as f:
        reader = csv.reader(f)
        for line in reader:
            logs.append(line)
        log_labels = logs.pop(0)

    for i in range(len(logs)):
        for j in range(3):
            img_path = logs[i][j]
            img_path = features_directory + 'IMG' + (img_path.split('IMG')[1]).strip()
            img = plt.imread(img_path)
            features.append(image_preprocessing(img))
            if j == 0:
                labels.append(float(logs[i][3]))
            elif j == 1:
                labels.append(float(logs[i][3]) + delta)
            else:
                labels.append(float(logs[i][3]) - delta)
    return features, labels





delta = 0.2
features,labels = Load_training(delta)

features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')

with open('features','wb') as f:
    pickle.dump(features,f,protocol=4)
with open('labels','wb') as f:
    pickle.dump(labels,f,protocol=4)


(24108, 40, 40)

In [ ]:

from tensorflow.keras.optimizers import Adam,SGD
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt

from keras import layers

from keras import models

from keras.layers import (Input, Dense, Activation, ZeroPadding2D,
BatchNormalization, Flatten, Conv2D, concatenate, Lambda)

from keras.layers import (AveragePooling2D, MaxPooling2D, Dropout,
GlobalMaxPooling2D, GlobalAveragePooling2D)

from keras.models import Model, load_model
from keras import regularizers, optimizers

from sklearn.model_selection import train_test_split



path = '../content/data/'
path = os.path.join(path,'driving_log.csv')

data_frame = pd.read_csv(path)
center = data_frame[data_frame.columns[0]].values
left = data_frame[data_frame.columns[1]].values
right = data_frame[data_frame.columns[2]].values
steering = data_frame[data_frame.columns[3]].values

no_of_examples = len(steering)
print(no_of_examples)

def random_flip(image, steering_angle):

    image = cv2.flip(image, 1)
    steering_angle = -steering_angle

    return image, steering_angle


train_x = []
train_y = []

img_folder = '../content/data/IMG'
stear_adjust_factor = 0.2
IMAGE_HEIGHT = 100
IMAGE_WIDTH = 100

for i in range(no_of_examples):

    for choice in range(3):

        if choice == 0: #Center
            img = cv2.imread(os.path.join(img_folder,center[i].split('IMG/')[1]))
            steering_angle = steering[i]

        elif choice == 1: #Left
            img = cv2.imread(os.path.join(img_folder,left[i].split('IMG/')[1]))
            steering_angle = steering[i] + stear_adjust_factor

        elif choice == 2: #Right
            img = cv2.imread(os.path.join(img_folder,right[i].split('IMG/')[1]))
            steering_angle = steering[i] - stear_adjust_factor

        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,1]
        img = cv2.resize(img,(IMAGE_HEIGHT,IMAGE_WIDTH))

        train_x.append(img)
        train_y.append(steering_angle)

        flip_img,steering_angle = random_flip(img,steering_angle)

        train_x.append(flip_img)
        train_y.append(steering_angle)


train_x = np.array(train_x)
train_x = np.reshape(train_x,[train_x.shape[0],train_x.shape[1],train_x.shape[2],1])

train_y = np.array(train_y)
train_y = np.reshape(train_y,[train_y.shape[0],1])

print(train_x.shape)
print(train_y.shape)

x_train,x_test,y_train,y_test = train_test_split(train_x,train_y,random_state=42,test_size=.20)


def model(height,width):

    x_input = Input(shape=(height,width,1))

    x = Lambda(lambda x: x/127.5-1.0)(x_input)

    x = Conv2D(32,(3,3),activation='relu',padding='same')(x_input)

    x = Conv2D(32,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)

    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)

    x = Conv2D(64,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)

    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)

    x = Conv2D(128,(3,3),activation='relu',padding='same')(x)
    x = MaxPooling2D((2,2),padding='valid')(x)

    x = Flatten()(x)
    x = Dropout(0.5)(x)

    x = BatchNormalization()(x)
    x = Dense(512)(x)
    x = Dense(256)(x)
    x = Dense(64)(x)
    x = Dense(1)(x)

    model = Model(inputs=x_input,outputs=x,name='model')

    return model

model = model(IMAGE_HEIGHT,IMAGE_WIDTH)
print(model.summary())

opt = tf.keras.optimizers.Adam(lr=0.0001)
model.compile(loss='mse',
             optimizer=opt,
             metrics=['mse'])

model.summary()
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=32,epochs=10)

model.save('../content/')

pre2=model.evaluate(y_train,y_test,batch_size = 64)

def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,100])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,100])
  plt.legend()
  plt.show()


plot_history(history)
print("Accuracy of model on test data is: ",pre2[1]*100)

8036
(48216, 100, 100, 1)
(48216, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 50, 50, 64)        36928     
                        